## Load Model

Initialize hybrid model, diffusion components, and memory system

## Model Path Configuration

Configure your model checkpoint path and experiment settings below.

**Available Options:**
- **LT-NWM_L model**: Use if you have trained the full hybrid model
- **CDiT-L 100k parameters**: Use existing checkpoint for testing
- **Custom path**: Specify your own model file location

In [ ]:
# ===== Model Path Configuration =====
# Please modify the following paths according to your actual situation

# Option 1: Use trained LT-NWM_L model (if available)
# MODEL_PATH = 'logs/lt-nwm_l/checkpoints/latest.pth.tar'

# Option 2: Use existing CDiT-L 100k parameters (for testing)
MODEL_PATH = 'logs/nwm_cdit_xl/cdit_l_100000.pth.tar'

# Option 3: Other custom path
# MODEL_PATH = 'path/to/your/model.pth.tar'

# Experiment configuration name
EXP_NAME = 'hybrid_l40s_inference_memory'  # Use hybrid configuration

# Verify if model file exists
import os
if os.path.exists(MODEL_PATH):
    print(f"✅ Model file found: {MODEL_PATH}")
    # Get file size
    file_size = os.path.getsize(MODEL_PATH) / (1024**3)  # GB
    print(f"📁 File size: {file_size:.2f} GB")
else:
    print(f"❌ Model file not found: {MODEL_PATH}")
    print("Please check the path or train the model")
    
    # List possible model files
    possible_paths = [
        'logs/nwm_cdit_xl/checkpoints/',
        'logs/lt-nwm_l/checkpoints/',
        'logs/',
        'checkpoints/'
    ]
    
    print("\n🔍 Searching for possible model files:")
    for path in possible_paths:
        if os.path.exists(path):
            print(f"📂 {path}:")
            try:
                files = [f for f in os.listdir(path) if f.endswith('.pth.tar') or f.endswith('.pth')]
                if files:
                    for file in sorted(files)[:5]:  # Show first 5 files
                        full_path = os.path.join(path, file)
                        size = os.path.getsize(full_path) / (1024**2)  # MB
                        print(f"   - {file} ({size:.1f} MB)")
                else:
                    print("   (No .pth or .pth.tar files found)")
            except PermissionError:
                print("   (Cannot access this directory)")
        else:
            print(f"📂 {path}: (Directory does not exist)")

print(f"\nCurrent configuration:")
print(f"🎯 Experiment name: {EXP_NAME}")
print(f"📄 Model path: {MODEL_PATH}")

## Hybrid Model Configuration

Display current hybrid model configuration information and system status

In [ ]:
import yaml
import torch
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
from PIL import Image
import requests
from io import BytesIO
import numpy as np
from IPython.display import display, HTML
import ipywidgets as widgets
# Fix: Use correct AutoencoderKL import path
from diffusers.models.autoencoders.autoencoder_kl import AutoencoderKL

from diffusion import create_diffusion
import sys
import os
# Fix: Use current working directory instead of __file__ in notebook
current_dir = os.getcwd()
sys.path.append(os.path.join(current_dir, 'scripts'))
from scripts.isolated_nwm_infer import model_forward_wrapper
from misc import transform
# Modified: Use hybrid_models instead of models
from hybrid_models import HybridCDiT_models
# Removed unused TrainingDataset import

# Check if model path and experiment name are defined
try:
    print(f"Using model path: {MODEL_PATH}")
    print(f"Using experiment config: {EXP_NAME}")
except NameError:
    print("❌ Please run the model path configuration cell above first!")
    raise Exception("MODEL_PATH and EXP_NAME are not defined, please run configuration cell first")

with open("config/data_config.yaml", "r") as f:
    default_config = yaml.safe_load(f)
config = default_config

# Modified: Use hybrid configuration file
with open(f'config/{EXP_NAME}.yaml', "r") as f:
    user_config = yaml.safe_load(f)
config.update(user_config)
latent_size = config['image_size'] // 8

print("Loading hybrid model with memory...")
# Modified: Use HybridCDiT_models and enable memory functionality
model_name = config.get('model', 'CDiT-L/2')
# Convert CDiT model name to HybridCDiT model name
if 'CDiT-' in model_name:
    hybrid_model_name = model_name.replace('CDiT-', 'HybridCDiT-')
else:
    hybrid_model_name = 'HybridCDiT-L/2'  # Default to L/2

# Fix: Remove context_size parameter as it's already set in model definition
model = HybridCDiT_models[hybrid_model_name](
    input_size=latent_size, 
    memory_enabled=config.get('memory_enabled', True),
    memory_buffer_size=config.get('memory_buffer_size', 100),
    memory_layers=config.get('memory_layers', None)
)

print(f"🔄 Loading model parameters: {MODEL_PATH}")
ckp = torch.load(MODEL_PATH, map_location='cpu', weights_only=False)
missing_keys, unexpected_keys = model.load_state_dict(ckp["ema"], strict=False)

if missing_keys:
    print(f"⚠️  Missing keys: {len(missing_keys)} items")
    if len(missing_keys) <= 10:
        for key in missing_keys:
            print(f"   - {key}")
    else:
        print(f"   (Showing first 10): {missing_keys[:10]}")

if unexpected_keys:
    print(f"⚠️  Unexpected keys: {len(unexpected_keys)} items")
    if len(unexpected_keys) <= 10:
        for key in unexpected_keys:
            print(f"   - {key}")
    else:
        print(f"   (Showing first 10): {unexpected_keys[:10]}")

if not missing_keys and not unexpected_keys:
    print("✅ Model parameters match perfectly")

model.eval()
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")
model.to(device)

# Only compile if CUDA is available (compilation may cause issues on CPU)
if torch.cuda.is_available():
    model = torch.compile(model)
    print("✅ Model compiled for GPU")
else:
    print("⚠️  Running on CPU - model compilation skipped")

# Initialize memory system
if hasattr(model, 'reset_memory'):
    model.reset_memory()
    print("✅ Memory system initialized")
else:
    print("⚠️  Model does not have memory system methods")

diffusion = create_diffusion(str(250))
vae = AutoencoderKL.from_pretrained(f"stabilityai/sd-vae-ft-ema").to(device)
latent_size = config['image_size'] // 8

print("🎉 Model loading completed!")

## ? CPU Simulation Mode - Skip GPU Model Loading

Create mock versions to test code logic without actually loading large models, suitable for systems without GPU or insufficient memory

In [ ]:
"""
CPU Simulation Mode - Do not load actual models, create mock objects for testing
Suitable for systems without GPU or insufficient memory
"""

import yaml
import torch
import numpy as np
from PIL import Image
import requests
from io import BytesIO
from IPython.display import display, HTML
import ipywidgets as widgets

# Check model path configuration
try:
    print(f"✅ Using model path: {MODEL_PATH}")
    print(f"✅ Using experiment config: {EXP_NAME}")
except NameError:
    print("❌ Please run the model path configuration cell above first!")
    raise Exception("MODEL_PATH and EXP_NAME are not defined")

# Load configuration files
with open("config/data_config.yaml", "r") as f:
    default_config = yaml.safe_load(f)
config = default_config

try:
    with open(f'config/{EXP_NAME}.yaml', "r") as f:
        user_config = yaml.safe_load(f)
    config.update(user_config)
    print(f"✅ Config loaded from: config/{EXP_NAME}.yaml")
except FileNotFoundError:
    print(f"⚠️  Config file not found: config/{EXP_NAME}.yaml, using defaults")

latent_size = config['image_size'] // 8

# Create mock model classes
class MockHybridModel:
    """Mock hybrid model class for testing code logic"""
    
    def __init__(self, latent_size, memory_enabled=True, memory_buffer_size=100):
        self.latent_size = latent_size
        self.memory_enabled = memory_enabled
        self.device = 'cpu'
        
        # Mock memory buffer
        if memory_enabled:
            self.memory_buffer = MockMemoryBuffer(memory_buffer_size)
        else:
            self.memory_buffer = None
    
    def parameters(self):
        """Mock parameters method, returns fake parameters for testing"""
        # Create some fake parameter tensors
        fake_params = [
            torch.randn(1000, 512),  # Fake weight matrix
            torch.randn(512),        # Fake bias
            torch.randn(2048, 1024), # More fake parameters
        ]
        return fake_params
    
    def eval(self):
        """Set to evaluation mode"""
        return self
    
    def to(self, device):
        """Move to device"""
        self.device = device
        return self
    
    def forward(self, *args, **kwargs):
        """Mock forward pass"""
        # Return random noise as output
        batch_size = 1
        return torch.randn(batch_size, 4, self.latent_size, self.latent_size)
    
    def reset_memory(self):
        """Reset memory system"""
        if self.memory_buffer:
            self.memory_buffer.reset_memory()
    
    def store_frame_in_memory(self, frame, pose, action):
        """Store frame in memory"""
        if self.memory_buffer:
            self.memory_buffer.add_frame(frame, pose, action)
    
    def get_memory_stats(self):
        """Get memory statistics"""
        if self.memory_buffer:
            return self.memory_buffer.get_stats()
        return {'memory_count': 0, 'max_capacity': 0, 'usage_ratio': 0.0}

class MockMemoryBuffer:
    """Mock memory buffer"""
    
    def __init__(self, max_size=100):
        self.max_size = max_size
        self.frames = []
        self.poses = []
        self.actions = []
        self.scores = []
        self.usage_counts = []
        self.unused_steps = []
        self.frame_indices = []
        self.current_index = 0
    
    def add_frame(self, frame, pose, action):
        """Add new frame"""
        if len(self.frames) < self.max_size:
            self.frames.append(frame)
            self.poses.append(pose)
            self.actions.append(action)
            self.scores.append(np.random.uniform(0.5, 1.0))  # Random score
            self.usage_counts.append(0)
            self.unused_steps.append(0)
            self.frame_indices.append(self.current_index)
        self.current_index += 1
    
    def reset_memory(self):
        """Reset memory"""
        self.frames.clear()
        self.poses.clear()
        self.actions.clear()
        self.scores.clear()
        self.usage_counts.clear()
        self.unused_steps.clear()
        self.frame_indices.clear()
        self.current_index = 0
    
    def get_stats(self):
        """Get statistics"""
        return {
            'memory_count': len(self.frames),
            'max_capacity': self.max_size,
            'usage_ratio': len(self.frames) / self.max_size if self.max_size > 0 else 0.0
        }

class MockDiffusion:
    """Mock diffusion model"""
    
    def p_sample_loop(self, model_fn, shape, noise, **kwargs):
        """Mock sampling loop"""
        # Return randomly generated samples
        return torch.randn(*shape)

class MockVAE:
    """Mock VAE model"""
    
    def __init__(self, device='cpu'):
        self.device = device
    
    def encode(self, x):
        """Mock encoding"""
        class MockLatentDist:
            def sample(self):
                # Return mock latent representation
                return torch.randn(x.shape[0], 4, x.shape[2]//8, x.shape[3]//8)
        
        class MockOutput:
            latent_dist = MockLatentDist()
        
        return MockOutput()
    
    def decode(self, z):
        """Mock decoding"""
        class MockSample:
            @property
            def sample(self):
                # Decode from latent space back to image space
                return torch.randn(z.shape[0], 3, z.shape[2]*8, z.shape[3]*8)
        
        return MockSample()
    
    def to(self, device):
        self.device = device
        return self

print("🏗️  Creating mock models for CPU testing...")

# Create mock model and components
model_name = config.get('model', 'CDiT-L/2')
hybrid_model_name = model_name.replace('CDiT-', 'HybridCDiT-') if 'CDiT-' in model_name else 'HybridCDiT-L/2'

model = MockHybridModel(
    latent_size=latent_size,
    memory_enabled=config.get('memory_enabled', True),
    memory_buffer_size=config.get('memory_buffer_size', 100)
)

# Mock device
device = 'cpu'  # Force use CPU
print(f"🖥️  Using device: {device}")

# Move model to device
model.to(device)

# Initialize memory system
if hasattr(model, 'reset_memory'):
    model.reset_memory()
    print("✅ Mock memory system initialized")

# Create mock diffusion and VAE
diffusion = MockDiffusion()
vae = MockVAE(device)

print("🎉 Mock model setup completed!")
print("\n📝 Note: This is a simulation version for testing code logic")
print("   - All outputs are randomly generated")
print("   - Will not produce real navigation results")
print("   - But can verify code structure and interface correctness")

## System Requirements & Performance Notes

**Hardware Requirements:**
- **Recommended**: NVIDIA GPU with 8GB+ VRAM
- **Minimum**: 16GB+ system RAM for CPU fallback

**Automatic Optimizations:**
- GPU detection and device switching
- Conditional model compilation (GPU only)
- Mixed precision inference (when supported)

**Known Limitations:**
- CPU inference may be extremely slow
- Large memory models may cause kernel crashes on limited hardware

In [ ]:
# Check if model and configuration are loaded
try:
    # Type hints to help Pylance recognize variable types
    from typing import Any, Dict, Union
    
    # Verify necessary variables exist
    if 'model' not in globals():
        raise NameError("model is not defined - please run the model loading cell first")
    if 'config' not in globals():
        raise NameError("config is not defined - please run the model loading cell first")
    
    # Display Hybrid Model Configuration Information
    print("=== Hybrid Model Configuration ===")
    print(f"Experiment name: {EXP_NAME}")
    print(f"Model path: {MODEL_PATH}")
    
    # Safely get hybrid_model_name
    if 'hybrid_model_name' in globals():
        print(f"Model type: {hybrid_model_name}")
    else:
        print("Model type: Mock Model (CPU Testing)")
    
    print(f"Input size: {latent_size}")
    print(f"Context size: {config.get('context_size', 'Not specified')}")
    print(f"Memory enabled: {config.get('memory_enabled', True)}")
    print(f"Memory buffer size: {config.get('memory_buffer_size', 100)}")
    print(f"Memory layers: {config.get('memory_layers', 'Auto')}")
    print(f"Adaptive memory weights: {config.get('adaptive_memory_weights', True)}")

    # Display model parameter count (safe handling)
    try:
        params = list(model.parameters())
        total_params = sum(p.numel() for p in params)
        trainable_params = sum(p.numel() for p in params if hasattr(p, 'requires_grad') and p.requires_grad)
        print(f"\nModel Parameters:")
        print(f"Total parameters: {total_params:,}")
        print(f"Trainable parameters: {trainable_params:,}")
    except Exception as param_error:
        print(f"\nModel Parameters: Unable to calculate ({param_error})")

    # Display memory system configuration (safe handling)
    try:
        if hasattr(model, 'memory_buffer') and getattr(model, 'memory_buffer', None) is not None:
            memory_buffer = getattr(model, 'memory_buffer')
            print(f"\nMemory System:")
            print(f"Max buffer size: {getattr(memory_buffer, 'max_size', 'Unknown')}")
            
            # Safely get frames list length
            frames = getattr(memory_buffer, 'frames', [])
            print(f"Current memory count: {len(frames) if frames is not None else 0}")
            
            if hasattr(memory_buffer, 'adaptive_scorer'):
                print("Adaptive scoring system: Enabled")
        else:
            print("\nMemory System: Not enabled")
    except Exception as memory_error:
        print(f"\nMemory System: Error accessing memory system ({memory_error})")

    # Display retrieval weights configuration (if any)
    if isinstance(config, dict) and 'retrieval_weights' in config:
        print(f"\nRetrieval weights configuration:")
        retrieval_weights = config['retrieval_weights']
        if isinstance(retrieval_weights, dict):
            for key, value in retrieval_weights.items():
                print(f"  {key}: {value}")

    print("\n=== Configuration loading completed ===\n")
    
except NameError as e:
    print(f"❌ Error: {e}")
    print("Please run the model loading cell above first!")
except Exception as e:
    print(f"❌ Error displaying configuration: {e}")
    print("Some configuration details may be unavailable")

## Choose Starting Image

Select an initial environment image from the gallery below to begin navigation

In [ ]:
def url_to_pil_image(url):
    response = requests.get(url)
    img = Image.open(BytesIO(response.content)).convert('RGB')
    return img

def load_internet_image(url):
    from torchvision import transforms
    _transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5], inplace=True),
    ])
    img = url_to_pil_image(url)
    x_start = _transform(img)
    return x_start.unsqueeze(0).expand(config['context_size'], x_start.shape[0], x_start.shape[1], x_start.shape[2])

# Jupyter Notebook Cell

# List of image links
image_links = [
    # Official dataset images
    #'https://raw.githubusercontent.com/amirbar/amirbar.github.io/refs/heads/master/images/recon.png',
    #'https://raw.githubusercontent.com/amirbar/amirbar.github.io/refs/heads/master/images/scand.png',
    #'https://raw.githubusercontent.com/amirbar/amirbar.github.io/refs/heads/master/images/sacson.png',
    #'https://raw.githubusercontent.com/amirbar/amirbar.github.io/refs/heads/master/images/tartan.png',
    
    # Custom warehouse images 
    # Known images different from official website
    #'https://raw.githubusercontent.com/FBI-openup/nwm/main/warehouse%20Images/frame.png',
    
    # Unknown warehouse environments (for testing generalization)
    'https://raw.githubusercontent.com/FBI-openup/nwm/main/warehouse%20Images/realistic-warehouse-photo1.png',
    'https://raw.githubusercontent.com/FBI-openup/nwm/main/warehouse%20Images/warehouse2.png',
    #'https://raw.githubusercontent.com/FBI-openup/nwm/main/warehouse%20Images/obstracle+warehouse3.jpg',
    #'https://raw.githubusercontent.com/FBI-openup/nwm/main/warehouse%20Images/Resized.jpg',
    #'https://raw.githubusercontent.com/FBI-openup/nwm/main/warehouse%20Images/realwarehouse.png',
    #'https://raw.githubusercontent.com/FBI-openup/nwm/main/warehouse%20Images/Low.png',
	
    # Unknown environments from internet
    #'https://raw.githubusercontent.com/FBI-openup/nwm/main/warehouse%20Images/livingroom.jpg',
   
    # Unknown environments from official website
    #'https://raw.githubusercontent.com/FBI-openup/nwm/main/warehouse%20Images/chateau.png',
    # Unknown from roboflow
    #'https://raw.githubusercontent.com/FBI-openup/nwm/main/warehouse%20Images/cornel.png'
]

# Output widget to hold the selected link
output = widgets.Output()
x_start_link = None  # This will hold the selected link

# Function to handle image click
def on_image_click(link):
    global x_start_link
    x_start_link = link
    with output:
        output.clear_output()
        print(f"Selected image link:\n{x_start_link}")

# Create HBox of images
image_buttons = []
for link in image_links:
    img = widgets.Button(
        description='click',
        layout=widgets.Layout(width='150px', height='20px', padding='0'),
        style={'button_color': 'lightgray'}
    )

    img._dom_classes += ('image-button',)
    img_link = link  # capture current link in closure

    def on_click(b, link=img_link):
        on_image_click(link)

    img.on_click(on_click)

    # Embed image using HTML style
    img_html = f'<img src="{link}" width="150px" height="150px">'
    img_html_widget = widgets.HTML(value=img_html)
    image_buttons.append(widgets.VBox([img_html_widget, img]))

# Display the gallery
display(widgets.HBox(image_buttons))
display(output)

## Interactive Navigation

Use the buttons below to control navigation and observe memory-enhanced predictions

In [ ]:
# Improved CPU-friendly interactive navigation system
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, Circle
import matplotlib.patches as mpatches

# Check necessary variables
if 'x_start_link' not in globals() or x_start_link is None:
    print("⚠️  Please select an image from the gallery above first!")
    x_start_link = 'https://raw.githubusercontent.com/FBI-openup/nwm/main/warehouse%20Images/realistic-warehouse-photo1.png'
    print(f"Using default image: {x_start_link}")

print(f"Selected starting image: {x_start_link}")

# Navigation command definitions
commands = {
    'Forward': [1, 0, 0],
    'Rotate Right': [0, 0, -0.5],
    'Rotate Left': [0, 0, 0.5],
    'Backward': [-1, 0, 0]
}

# Initialize state variables
navigation_state = {
    'current_pose': torch.zeros(4).to(device),  # [x, y, z, yaw]
    'step_count': 0,
    'trajectory': [(0, 0)],  # Record trajectory
    'actions_taken': [],
    'memory_usage': []
}

def mock_image_generation(action_name):
    """Generate mock navigation images"""
    # Create simple visualization
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    
    # Left: Mock generated environment image
    ax1.set_title(f"Generated View - Step {navigation_state['step_count']}")
    
    # Create simple warehouse environment simulation
    warehouse_bg = np.random.uniform(0.3, 0.7, (224, 224, 3))
    
    # Add some "shelves" and "corridors"
    for i in range(3):
        x_start = 40 + i * 60
        warehouse_bg[50:170, x_start:x_start+20] = [0.8, 0.6, 0.4]  # Shelf color
    
    # Add floor texture
    warehouse_bg[170:, :] = [0.5, 0.5, 0.6]
    
    # Add visual changes based on action
    if action_name == 'Forward':
        warehouse_bg[100:120, 100:120] = [1.0, 0.8, 0.8]  # Forward highlight
    elif 'Rotate' in action_name:
        warehouse_bg[80:140, 80:140] = warehouse_bg[80:140, 80:140] * 0.9  # Rotation effect
    
    ax1.imshow(warehouse_bg)
    ax1.set_xlabel(f"Action: {action_name}")
    ax1.axis('off')
    
    # Right: Trajectory and status display
    ax2.set_title("Navigation Trajectory & Memory Status")
    ax2.set_xlim(-5, 5)
    ax2.set_ylim(-5, 5)
    ax2.grid(True, alpha=0.3)
    
    # Draw trajectory
    if len(navigation_state['trajectory']) > 1:
        trajectory = np.array(navigation_state['trajectory'])
        ax2.plot(trajectory[:, 0], trajectory[:, 1], 'b-', alpha=0.7, linewidth=2, label='Path')
        ax2.scatter(trajectory[:, 0], trajectory[:, 1], c='blue', s=30, alpha=0.7)
    
    # Draw current position
    current_pos = navigation_state['trajectory'][-1]
    ax2.scatter(current_pos[0], current_pos[1], c='red', s=100, marker='o', label='Current Position')
    
    # Draw orientation
    yaw = navigation_state['current_pose'][3].item()
    dx = 0.5 * np.cos(yaw)
    dy = 0.5 * np.sin(yaw)
    ax2.arrow(current_pos[0], current_pos[1], dx, dy, head_width=0.2, head_length=0.2, fc='red', ec='red')
    
    # Add memory usage information
    memory_stats = navigation_state['memory_usage']
    if memory_stats:
        latest_memory = memory_stats[-1]
        ax2.text(0.02, 0.98, f"Memory: {latest_memory['memory_count']}/{latest_memory['max_capacity']}", 
                transform=ax2.transAxes, verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
    
    ax2.legend()
    ax2.set_xlabel('X Position')
    ax2.set_ylabel('Y Position')
    
    plt.tight_layout()
    plt.show()

def update_navigation_state(action_name):
    """Update navigation state"""
    action = commands[action_name]
    action_tensor = torch.tensor(action).to(device)
    
    # Update pose
    if action_name == 'Forward':
        navigation_state['current_pose'][0] += 0.5  # Move forward in x
    elif action_name == 'Backward':
        navigation_state['current_pose'][0] -= 0.5  # Move backward in x
    elif action_name == 'Rotate Left':
        navigation_state['current_pose'][3] += 0.3  # Rotate left (yaw)
    elif action_name == 'Rotate Right':
        navigation_state['current_pose'][3] -= 0.3  # Rotate right (yaw)
    
    # Update trajectory
    x, y = navigation_state['current_pose'][0].item(), navigation_state['current_pose'][1].item()
    navigation_state['trajectory'].append((x, y))
    
    # Update step count
    navigation_state['step_count'] += 1
    navigation_state['actions_taken'].append(action_name)
    
    # Simulate memory usage (add to mock memory buffer)
    if hasattr(model, 'memory_buffer'):
        # Create mock observation frame
        mock_frame = torch.randn(4, 28, 28)  # Mock latent features
        model.store_frame_in_memory(mock_frame, navigation_state['current_pose'], action_tensor)
        
        # Record memory state
        memory_stats = model.get_memory_stats()
        navigation_state['memory_usage'].append(memory_stats)

def reset_navigation():
    """Reset navigation state"""
    navigation_state['current_pose'] = torch.zeros(4).to(device)
    navigation_state['step_count'] = 0
    navigation_state['trajectory'] = [(0, 0)]
    navigation_state['actions_taken'] = []
    navigation_state['memory_usage'] = []
    
    if hasattr(model, 'reset_memory'):
        model.reset_memory()
    
    print("🔄 Navigation reset!")
    mock_image_generation("Reset")

# Create improved control interface
def create_navigation_controls():
    """Create navigation control interface"""
    output = widgets.Output()
    
    def on_action_click(action_name):
        with output:
            output.clear_output(wait=True)
            print(f"🎮 Action: {action_name}")
            update_navigation_state(action_name)
            mock_image_generation(action_name)
            
            # Display status information
            print(f"Step: {navigation_state['step_count']}")
            pose = navigation_state['current_pose']
            print(f"Position: x={pose[0]:.2f}, y={pose[1]:.2f}, yaw={pose[3]:.2f}")
            
            if navigation_state['memory_usage']:
                latest_memory = navigation_state['memory_usage'][-1]
                print(f"Memory: {latest_memory['memory_count']}/{latest_memory['max_capacity']} "
                      f"({latest_memory['usage_ratio']:.1%} full)")
    
    def on_reset_click(b):
        with output:
            output.clear_output()
            reset_navigation()
    
    # Create buttons
    buttons = []
    for action in ['Forward', 'Rotate Left', 'Rotate Right', 'Backward']:
        btn = widgets.Button(
            description=f"🎯 {action}",
            layout=widgets.Layout(width='150px', margin='2px'),
            style={'button_color': 'lightgreen'}
        )
        btn.on_click(lambda b, action=action: on_action_click(action))
        buttons.append(btn)
    
    reset_btn = widgets.Button(
        description="🔄 Reset",
        layout=widgets.Layout(width='150px', margin='2px'),
        style={'button_color': 'orange'}
    )
    reset_btn.on_click(on_reset_click)
    
    # Layout
    controls = widgets.VBox([
        widgets.HTML("<h3>🕹️ Navigation Controls (CPU Mock Mode)</h3>"),
        widgets.HBox(buttons[:2]),
        widgets.HBox(buttons[2:] + [reset_btn]),
        output
    ])
    
    return controls

# Initialize and display control interface
print("🚀 Initializing CPU-friendly navigation system...")
navigation_controls = create_navigation_controls()
display(navigation_controls)

# Display initial state
reset_navigation()

print("\n✅ Navigation system ready!")
print("💡 Tip: This is CPU simulation mode, all images are programmatically generated visualizations")

## Memory System Status

Monitor the current state and performance of the memory buffer system

In [ ]:
# Memory system status monitoring
def display_memory_status():
    """Display detailed memory system status"""
    if not hasattr(model, 'memory_buffer') or model.memory_buffer is None:
        print("Memory system not enabled")
        return
    
    buffer = model.memory_buffer
    print("=== Memory System Status ===")
    print(f"Current memory count: {len(buffer.frames)}/{buffer.max_size}")
    print(f"Capacity usage: {len(buffer.frames)/buffer.max_size:.1%}")
    
    if len(buffer.frames) > 0:
        print(f"Memory score range: {min(buffer.scores):.1f} - {max(buffer.scores):.1f}")
        print(f"Average usage count: {sum(buffer.usage_counts)/len(buffer.usage_counts):.1f}")
        
        # Display recent memories
        print("\nRecent 5 memories:")
        recent_indices = sorted(range(len(buffer.frame_indices)), 
                              key=lambda i: buffer.frame_indices[i], reverse=True)[:5]
        for i in recent_indices:
            print(f"  Frame {buffer.frame_indices[i]}: score={buffer.scores[i]:.1f}, "
                  f"used={buffer.usage_counts[i]}x, unused_steps={buffer.unused_steps[i]}")

# Create button to display memory status
status_button = widgets.Button(description="Show Memory Status", style={'button_color': 'lightblue'})
status_output = widgets.Output()

def show_status(b):
    with status_output:
        status_output.clear_output()
        display_memory_status()

status_button.on_click(show_status)
display(status_button)
display(status_output)

In [ ]:
# Improved memory system status monitoring
def display_memory_status():
    """Display detailed memory system status - compatible with mock models"""
    try:
        if not hasattr(model, 'memory_buffer') or getattr(model, 'memory_buffer', None) is None:
            print("❌ Memory system not enabled")
            return
        
        buffer = getattr(model, 'memory_buffer')
        print("=== Memory System Status ===")
        
        # Safely get memory information
        frames = getattr(buffer, 'frames', [])
        max_size = getattr(buffer, 'max_size', 0)
        
        print(f"Current memory count: {len(frames)}/{max_size}")
        print(f"Capacity usage: {len(frames)/max_size:.1%}" if max_size > 0 else "Capacity usage: N/A")
        
        if len(frames) > 0:
            # Safely get score information
            scores = getattr(buffer, 'scores', [])
            usage_counts = getattr(buffer, 'usage_counts', [])
            
            if scores:
                print(f"Memory score range: {min(scores):.1f} - {max(scores):.1f}")
            
            if usage_counts:
                avg_usage = sum(usage_counts)/len(usage_counts)
                print(f"Average usage count: {avg_usage:.1f}")
            
            # Display recent memories
            print("\nRecent memories:")
            frame_indices = getattr(buffer, 'frame_indices', list(range(len(frames))))
            
            # Show most recent 5 memory entries
            recent_count = min(5, len(frames))
            for i in range(recent_count):
                idx = len(frames) - 1 - i  # Start from newest
                frame_idx = frame_indices[idx] if idx < len(frame_indices) else idx
                score = scores[idx] if idx < len(scores) else 0.0
                usage = usage_counts[idx] if idx < len(usage_counts) else 0
                
                print(f"  Frame {frame_idx}: score={score:.1f}, used={usage}x")
        else:
            print("No memories stored yet")
            
    except Exception as e:
        print(f"❌ Error accessing memory system: {e}")
        print("Memory system may not be properly initialized")

# Create interactive button - improved version
def create_memory_status_widget():
    """Create memory status display widget"""
    status_button = widgets.Button(
        description="🔍 Show Memory Status", 
        style={'button_color': 'lightblue'},
        layout=widgets.Layout(width='200px')
    )
    status_output = widgets.Output()

    def show_status(b):
        with status_output:
            status_output.clear_output()
            display_memory_status()

    status_button.on_click(show_status)
    
    # Create description text
    info_text = widgets.HTML(
        value="<p><b>Memory System Monitor</b><br/>Click button to view current memory buffer status</p>"
    )
    
    return widgets.VBox([info_text, status_button, status_output])

# Display widget
memory_widget = create_memory_status_widget()
display(memory_widget)

print("✅ Memory status monitor created")
print("💡 Tip: In simulation mode, memory data is randomly generated for interface testing only")

## 🧪 Comprehensive Testing: File Path & Memory System Validation

Specifically test whether your modified file structure and memory mechanisms are correctly configured

# Navigation Video Generation

Generate an animated video from the recorded navigation sequence

In [ ]:

%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import animation
from IPython.display import HTML
import numpy as np
# np array with shape (frames, height, width, channels)
video = np.array(preds['video'])

fig = plt.figure()
im = plt.imshow(video[0,:,:,:])

plt.close() # this is required to not display the generated image

def init():
    im.set_data(video[0,:,:,:])

def animate(i):
    im.set_data(video[i,:,:,:])
    return im

anim = animation.FuncAnimation(fig, animate, init_func=init, frames=video.shape[0],
                               interval=500)
HTML(anim.to_html5_video())

In [ ]:
# Optional: load from dataset
# dataloaders = {}

# for dataset_name in config["datasets"]:
#     data_config = config["datasets"][dataset_name]
#     for data_split_type in ["test"]: #["test"]:
#         dataset = TrainingDataset(
#             data_folder=data_config["data_folder"],
#             data_split_folder=data_config[data_split_type],
#             dataset_name=dataset_name,
#             image_size=config["image_size"],
#             min_dist_cat=config["distance"]["min_dist_cat"],
#             max_dist_cat=config["distance"]["max_dist_cat"],
#             len_traj_pred=config["len_traj_pred"],
#             context_size=config["context_size"],
#             normalize=config["normalize"],
#             goals_per_obs=1,
#             transform=transform,
#             predefined_index=None,
#             traj_stride=1,
#         )
#         dataloaders[f"{dataset_name}_{data_split_type}"] = dataset
#         print(f"Dataset: {dataset_name} ({data_split_type}), size: {len(dataset)}")

# Load from dataset
# ds = dataloaders['recon_test'] # scand_test,
# x, _, _ = ds[np.random.randint(len(ds))]
# x_start = x[:config["context_size"]]

## 🔧 Notebook Cell Diagnostics & Repair

If you encounter issues such as cell backgrounds dimming or inability to edit cells, please use the methods below for repair

In [ ]:
"""
Notebook Cell State Repair Tool
Run this cell when encountering the following issues:
1. Code background dimming
2. Unable to edit cells
3. Cell locked after Copilot modifications
"""

import sys
import gc
from IPython.display import display, HTML, Javascript

def fix_notebook_state():
    """Fix notebook state issues"""
    print("🔧 Starting notebook state repair...")
    
    # 1. Memory cleanup
    gc.collect()
    print("✅ Memory cleanup completed")
    
    # 2. Check kernel state
    print(f"🐍 Python version: {sys.version}")
    print(f"💾 Memory usage: {sys.getsizeof(globals())} bytes")
    
    # 3. Reset IPython display settings
    try:
        from IPython.core.display import clear_output
        print("✅ IPython display system normal")
    except Exception as e:
        print(f"⚠️ IPython display system exception: {e}")
    
    # 4. Check variable state
    important_vars = ['MODEL_PATH', 'EXP_NAME', 'model', 'device', 'config']
    for var in important_vars:
        if var in globals():
            print(f"✅ {var}: Defined")
        else:
            print(f"❌ {var}: Not defined")
    
    print("\n🎯 Repair suggestions:")
    print("1. If cell background is dimmed, double-click the cell to enter edit mode")
    print("2. Press Ctrl+Z to undo recent modifications")
    print("3. Press Ctrl+Shift+P and search for 'Reload Window'")
    print("4. If issues persist, restart kernel (Ctrl+Shift+P -> 'Restart Kernel')")

# Run repair
fix_notebook_state()

# JavaScript code to fix frontend display issues
display(Javascript("""
// Fix cell display issues
console.log('Fixing cell state...');

// Remove possible locked state
document.querySelectorAll('.cell').forEach(cell => {
    cell.classList.remove('readonly', 'locked', 'disabled');
    cell.style.opacity = '1';
    cell.style.backgroundColor = '';
});

// Reactivate edit functionality
document.querySelectorAll('.jp-Cell').forEach(cell => {
    cell.style.opacity = '1';
    cell.style.filter = 'none';
});

console.log('Cell state repair completed');
"""))

print("\n✨ State repair completed! If issues persist, try the following operations:")

### 🚀 Manual Repair Steps

If automatic repair is ineffective, please follow these steps:

#### Method 1: Reactivate Cells
1. **Double-click the dimmed cell** - Force entry into edit mode
2. **Press `Esc` then `Enter`** - Exit and re-enter edit mode
3. **Press `Ctrl+Z`** - Undo recent modifications

#### Method 2: VS Code Command Palette
1. **Press `Ctrl+Shift+P`** to open command palette
2. Search and execute the following commands:
   - `Developer: Reload Window` - Reload window
   - `Jupyter: Restart Kernel` - Restart kernel
   - `Jupyter: Clear All Outputs` - Clear all outputs

#### Method 3: Kernel Restart
1. Click the **"Restart"** button at the top
2. Or press `Ctrl+Shift+P` → `Jupyter: Restart Kernel and Clear All Outputs`

#### Method 4: Check Extensions
1. Ensure the following extensions are enabled:
   - **Jupyter Extension Pack**
   - **GitHub Copilot**
   - **Python Extension**

#### Preventive Measures
- **Save frequently**: Use `Ctrl+S` often
- **Small modifications**: Avoid large code changes at once
- **Backup important code**: Copy important code blocks to text files

## 🧪 Comprehensive CPU Testing Suite

Integrated testing framework for validating file structure, configuration loading, memory system, and overall integration without requiring GPU resources

In [ ]:
"""
🧪 Comprehensive CPU Testing Suite
Integrated testing framework for systems without GPU
Tests: File structure, Configuration loading, Memory system, Integration validation
"""

import os
import yaml
import torch
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, Circle
import matplotlib.patches as mpatches
import ipywidgets as widgets
from IPython.display import display, HTML, Javascript
import gc
import sys

# ===========================
# 1. MOCK MODEL CLASSES
# ===========================

class MockHybridModel:
    """Mock hybrid model class for testing code logic"""
    
    def __init__(self, latent_size, memory_enabled=True, memory_buffer_size=100):
        self.latent_size = latent_size
        self.memory_enabled = memory_enabled
        self.device = 'cpu'
        
        # Mock memory buffer
        if memory_enabled:
            self.memory_buffer = MockMemoryBuffer(memory_buffer_size)
        else:
            self.memory_buffer = None
    
    def parameters(self):
        """Mock parameters method, returns fake parameters for testing"""
        fake_params = [
            torch.randn(1000, 512),  # Fake weight matrix
            torch.randn(512),        # Fake bias
            torch.randn(2048, 1024), # More fake parameters
        ]
        return fake_params
    
    def eval(self):
        """Set to evaluation mode"""
        return self
    
    def to(self, device):
        """Move to device"""
        self.device = device
        return self
    
    def forward(self, *args, **kwargs):
        """Mock forward pass"""
        batch_size = 1
        return torch.randn(batch_size, 4, self.latent_size, self.latent_size)
    
    def reset_memory(self):
        """Reset memory system"""
        if self.memory_buffer:
            self.memory_buffer.reset_memory()
    
    def store_frame_in_memory(self, frame, pose, action):
        """Store frame in memory"""
        if self.memory_buffer:
            self.memory_buffer.add_frame(frame, pose, action)
    
    def get_memory_stats(self):
        """Get memory statistics"""
        if self.memory_buffer:
            return self.memory_buffer.get_stats()
        return {'memory_count': 0, 'max_capacity': 0, 'usage_ratio': 0.0}

class MockMemoryBuffer:
    """Mock memory buffer"""
    
    def __init__(self, max_size=100):
        self.max_size = max_size
        self.frames = []
        self.poses = []
        self.actions = []
        self.scores = []
        self.usage_counts = []
        self.unused_steps = []
        self.frame_indices = []
        self.current_index = 0
    
    def add_frame(self, frame, pose, action):
        """Add new frame"""
        if len(self.frames) < self.max_size:
            self.frames.append(frame)
            self.poses.append(pose)
            self.actions.append(action)
            self.scores.append(np.random.uniform(0.5, 1.0))  # Random score
            self.usage_counts.append(0)
            self.unused_steps.append(0)
            self.frame_indices.append(self.current_index)
        self.current_index += 1
    
    def reset_memory(self):
        """Reset memory"""
        self.frames.clear()
        self.poses.clear()
        self.actions.clear()
        self.scores.clear()
        self.usage_counts.clear()
        self.unused_steps.clear()
        self.frame_indices.clear()
        self.current_index = 0
    
    def get_stats(self):
        """Get statistics"""
        return {
            'memory_count': len(self.frames),
            'max_capacity': self.max_size,
            'usage_ratio': len(self.frames) / self.max_size if self.max_size > 0 else 0.0
        }

class MockDiffusion:
    """Mock diffusion model"""
    
    def p_sample_loop(self, model_fn, shape, noise, **kwargs):
        """Mock sampling loop"""
        return torch.randn(*shape)

class MockVAE:
    """Mock VAE model"""
    
    def __init__(self, device='cpu'):
        self.device = device
    
    def encode(self, x):
        """Mock encoding"""
        class MockLatentDist:
            def sample(self):
                return torch.randn(x.shape[0], 4, x.shape[2]//8, x.shape[3]//8)
        
        class MockOutput:
            latent_dist = MockLatentDist()
        
        return MockOutput()
    
    def decode(self, z):
        """Mock decoding"""
        class MockSample:
            @property
            def sample(self):
                return torch.randn(z.shape[0], 3, z.shape[2]*8, z.shape[3]*8)
        
        return MockSample()
    
    def to(self, device):
        self.device = device
        return self

# ===========================
# 2. TESTING FUNCTIONS
# ===========================

def test_file_structure():
    """Test 1: Validate file path structure"""
    print("=" * 50)
    print("🗂️  File Structure Test")
    print("=" * 50)
    
    # Check critical directories
    critical_paths = {
        "Model file": MODEL_PATH,
        "Config directory": "config/",
        "Logs directory": "logs/",
        "Scripts directory": "scripts/",
        "Data directory": "data/",
        "Data splits directory": "data_splits/"
    }
    
    results = {}
    for name, path in critical_paths.items():
        exists = os.path.exists(path)
        results[name] = exists
        
        status = "✅" if exists else "❌"
        print(f"{status} {name}: {path}")
        
        if exists and os.path.isdir(path):
            try:
                items = os.listdir(path)
                print(f"   📁 Contains {len(items)} items")
                if len(items) <= 5:
                    for item in items[:3]:
                        print(f"      - {item}")
                else:
                    print(f"      - {items[0]}, {items[1]}, ... ({len(items)} total)")
            except PermissionError:
                print("   ⚠️  Cannot access directory contents")
        elif exists and os.path.isfile(path):
            size_mb = os.path.getsize(path) / (1024**2)
            print(f"   📄 File size: {size_mb:.1f} MB")
    
    print(f"\n📊 Results: {sum(results.values())}/{len(results)} paths exist")
    return results

def test_config_loading():
    """Test 2: Configuration file loading"""
    print("\n" + "=" * 50)
    print("⚙️  Configuration File Loading Test")
    print("=" * 50)
    
    config_files = [
        "config/data_config.yaml",
        f"config/{EXP_NAME}.yaml",
        "config/data_hyperparams_plan.yaml",
        "config/eval_config.yaml"
    ]
    
    config_results = {}
    
    for config_file in config_files:
        try:
            if os.path.exists(config_file):
                with open(config_file, 'r') as f:
                    config_data = yaml.safe_load(f)
                
                print(f"✅ {config_file}")
                print(f"   📝 Contains {len(config_data)} configuration items")
                
                # Show important configurations
                important_keys = ['image_size', 'context_size', 'memory_enabled', 'memory_buffer_size']
                for key in important_keys:
                    if key in config_data:
                        print(f"   - {key}: {config_data[key]}")
                
                config_results[config_file] = True
            else:
                print(f"❌ {config_file} - File does not exist")
                config_results[config_file] = False
                
        except Exception as e:
            print(f"⚠️  {config_file} - Loading error: {e}")
            config_results[config_file] = False
    
    return config_results

def test_memory_storage_mechanism():
    """Test 3: Memory storage mechanism"""
    print("\n" + "=" * 50)
    print("🧠 Memory Storage Mechanism Test")
    print("=" * 50)
    
    # Reset memory buffer
    model.reset_memory()
    initial_count = len(model.memory_buffer.frames)
    print(f"Initial memory count: {initial_count}")
    
    # Simulate storing several observation frames
    test_frames = []
    test_poses = []
    test_actions = []
    
    print("\n📝 Storing test frames...")
    for i in range(5):
        # Create mock observation frame (latent features)
        frame = torch.randn(4, latent_size, latent_size)
        pose = torch.tensor([i * 0.5, 0.0, 0.0, i * 0.1])  # x, y, z, yaw
        action = torch.tensor([1.0, 0.0, 0.0])  # Forward action
        
        print(f"Storing frame {i+1}: pose=({pose[0]:.1f}, {pose[1]:.1f}, {pose[3]:.1f})")
        model.store_frame_in_memory(frame, pose, action)
        
        test_frames.append(frame)
        test_poses.append(pose)
        test_actions.append(action)
    
    # Verify storage results
    final_count = len(model.memory_buffer.frames)
    print(f"\n📊 Storage results:")
    print(f"   Before: {initial_count} frames")
    print(f"   After: {final_count} frames")
    print(f"   Added: {final_count - initial_count} frames")
    
    # Check memory buffer contents
    buffer = model.memory_buffer
    if hasattr(buffer, 'scores') and len(buffer.scores) > 0:
        print(f"   Score range: {min(buffer.scores):.2f} - {max(buffer.scores):.2f}")
    
    return {
        'initial_count': initial_count,
        'final_count': final_count,
        'frames_added': final_count - initial_count
    }

def test_memory_retrieval_criteria():
    """Test 4: Memory retrieval criteria"""
    print("\n" + "=" * 50)
    print("🎯 Memory Retrieval Criteria Test")
    print("=" * 50)
    
    # Get current configured retrieval weights
    retrieval_config = config.get('retrieval_weights', {})
    
    print("📋 Configured retrieval criteria:")
    total_weight = sum(retrieval_config.values())
    print(f"Weight sum: {total_weight} {'✅' if abs(total_weight - 1.0) < 0.01 else '⚠️ Not equal to 1.0'}")
    
    for criterion, weight in retrieval_config.items():
        percentage = weight * 100
        importance = "High" if weight > 0.3 else "Medium" if weight > 0.15 else "Low"
        print(f"   • {criterion}: {weight} ({percentage:.1f}%) - {importance} importance")
    
    # Simulate memory retrieval process
    buffer = model.memory_buffer
    
    if len(buffer.frames) > 0:
        print("\n🔍 Memory retrieval simulation:")
        
        # Current query
        current_pose = torch.tensor([2.0, 0.0, 0.0, 0.2])
        current_action = torch.tensor([1.0, 0.0, 0.0])
        
        print(f"Query position: ({current_pose[0]:.1f}, {current_pose[1]:.1f}, {current_pose[3]:.1f})")
        
        # Calculate similarity scores for each memory
        similarities = []
        for i, (stored_pose, stored_action) in enumerate(zip(buffer.poses[:5], buffer.actions[:5])):
            # Spatial similarity (closer distance = higher similarity)
            spatial_dist = torch.norm(current_pose[:3] - stored_pose[:3]).item()
            spatial_sim = 1.0 / (1.0 + spatial_dist)
            
            # Action similarity (cosine similarity)
            action_sim = torch.cosine_similarity(
                current_action.unsqueeze(0), 
                stored_action.unsqueeze(0)
            ).item()
            
            # Memory score (stored score)
            memory_score = buffer.scores[i] if i < len(buffer.scores) else 0.5
            
            # Usage frequency (less used = higher weight)
            usage_count = buffer.usage_counts[i] if i < len(buffer.usage_counts) else 0
            usage_weight = 1.0 / (1.0 + usage_count)
            
            # Combined score
            total_score = (
                retrieval_config.get('spatial_weight', 0.4) * spatial_sim +
                retrieval_config.get('action_weight', 0.3) * action_sim +
                retrieval_config.get('memory_weight', 0.2) * memory_score +
                retrieval_config.get('usage_weight', 0.1) * usage_weight
            )
            
            similarities.append({
                'index': i,
                'spatial_sim': spatial_sim,
                'action_sim': action_sim,
                'memory_score': memory_score,
                'usage_weight': usage_weight,
                'total_score': total_score
            })
            
            print(f"   Memory{i}: spatial={spatial_sim:.2f}, action={action_sim:.2f}, "
                  f"memory={memory_score:.2f}, usage={usage_weight:.2f} → total={total_score:.2f}")
        
        # Sort and show most relevant memories
        similarities.sort(key=lambda x: x['total_score'], reverse=True)
        
        print(f"\n🏆 Most relevant memories:")
        for i, sim in enumerate(similarities[:3]):
            print(f"   Rank {i+1}: Memory{sim['index']} (score: {sim['total_score']:.3f})")
    
    else:
        print("⚠️  Memory buffer is empty, cannot test retrieval")
    
    return retrieval_config

def test_integration():
    """Test 5: Integration test"""
    print("\n" + "=" * 50)
    print("🔗 Integration Test")
    print("=" * 50)
    
    print("📊 Environment status:")
    print(f"   Python device: {device}")
    print(f"   Model type: {type(model).__name__}")
    print(f"   Diffusion model: {type(diffusion).__name__}")
    print(f"   VAE model: {type(vae).__name__}")
    print(f"   Latent size: {latent_size}")
    
    # Test model call chain
    print("\n🔄 Model call chain test:")
    try:
        # Simulate complete inference
        mock_observation = torch.randn(1, 4, 3, 224, 224)  # batch, time, channels, height, width
        mock_action = torch.tensor([[1.0, 0.0, 0.0]])
        mock_pose = torch.zeros(4)
        
        print("   1. VAE encoding...")
        encoded = vae.encode(mock_observation.flatten(0, 1))
        latent = encoded.latent_dist.sample()
        print(f"      ✅ Latent shape: {latent.shape}")
        
        print("   2. Diffusion sampling...")
        noise = torch.randn_like(latent)
        denoised = diffusion.p_sample_loop(
            model.forward, 
            noise.shape, 
            noise
        )
        print(f"      ✅ Denoised shape: {denoised.shape}")
        
        print("   3. VAE decoding...")
        decoded = vae.decode(denoised)
        output = decoded.sample
        print(f"      ✅ Output shape: {output.shape}")
        
        print("   4. Memory update...")
        model.store_frame_in_memory(latent[0], mock_pose, mock_action[0])
        memory_stats = model.get_memory_stats()
        print(f"      ✅ Memory status: {memory_stats['memory_count']}/{memory_stats['max_capacity']}")
        
        return True
        
    except Exception as e:
        print(f"   ❌ Integration test failed: {e}")
        return False

def create_mock_navigation_demo():
    """Create mock navigation demonstration"""
    print("\n" + "=" * 50)
    print("🎮 Mock Navigation Demo")
    print("=" * 50)
    
    # Navigation commands
    commands = {
        'Forward': [1, 0, 0],
        'Rotate Right': [0, 0, -0.5],
        'Rotate Left': [0, 0, 0.5],
        'Backward': [-1, 0, 0]
    }
    
    # Initialize navigation state
    navigation_state = {
        'current_pose': torch.zeros(4).to(device),  # [x, y, z, yaw]
        'step_count': 0,
        'trajectory': [(0, 0)],
        'actions_taken': [],
        'memory_usage': []
    }
    
    def mock_navigation_step(action_name):
        """Simulate one navigation step"""
        action = commands[action_name]
        action_tensor = torch.tensor(action).to(device)
        
        # Update pose
        if action_name == 'Forward':
            navigation_state['current_pose'][0] += 0.5
        elif action_name == 'Backward':
            navigation_state['current_pose'][0] -= 0.5
        elif action_name == 'Rotate Left':
            navigation_state['current_pose'][3] += 0.3
        elif action_name == 'Rotate Right':
            navigation_state['current_pose'][3] -= 0.3
        
        # Update trajectory
        x, y = navigation_state['current_pose'][0].item(), navigation_state['current_pose'][1].item()
        navigation_state['trajectory'].append((x, y))
        
        # Update step count
        navigation_state['step_count'] += 1
        navigation_state['actions_taken'].append(action_name)
        
        # Store in memory
        mock_frame = torch.randn(4, 28, 28)
        model.store_frame_in_memory(mock_frame, navigation_state['current_pose'], action_tensor)
        
        # Record memory state
        memory_stats = model.get_memory_stats()
        navigation_state['memory_usage'].append(memory_stats)
        
        return navigation_state
    
    # Simulate sequence of actions
    action_sequence = ['Forward', 'Forward', 'Rotate Right', 'Forward', 'Rotate Left', 'Forward']
    
    print("🎯 Simulating navigation sequence:")
    for i, action in enumerate(action_sequence):
        state = mock_navigation_step(action)
        pose = state['current_pose']
        memory = state['memory_usage'][-1]
        
        print(f"   Step {i+1}: {action}")
        print(f"      Position: x={pose[0]:.1f}, y={pose[1]:.1f}, yaw={pose[3]:.1f}")
        print(f"      Memory: {memory['memory_count']}/{memory['max_capacity']} ({memory['usage_ratio']:.1%})")
    
    # Create simple visualization
    fig, ax = plt.subplots(1, 1, figsize=(8, 6))
    ax.set_title("Mock Navigation Trajectory")
    ax.set_xlim(-1, 4)
    ax.set_ylim(-1, 2)
    ax.grid(True, alpha=0.3)
    
    # Plot trajectory
    trajectory = np.array(navigation_state['trajectory'])
    ax.plot(trajectory[:, 0], trajectory[:, 1], 'b-', linewidth=2, marker='o', markersize=4)
    
    # Mark start and end
    ax.scatter(trajectory[0, 0], trajectory[0, 1], c='green', s=100, marker='s', label='Start')
    ax.scatter(trajectory[-1, 0], trajectory[-1, 1], c='red', s=100, marker='*', label='End')
    
    # Add action annotations
    for i, action in enumerate(action_sequence):
        if i < len(trajectory) - 1:
            ax.annotate(f"{i+1}:{action[:3]}", 
                       (trajectory[i+1, 0], trajectory[i+1, 1]),
                       xytext=(5, 5), textcoords='offset points', fontsize=8)
    
    ax.legend()
    ax.set_xlabel('X Position')
    ax.set_ylabel('Y Position')
    plt.tight_layout()
    plt.show()
    
    return navigation_state

def fix_notebook_state():
    """Fix notebook state issues"""
    print("\n" + "=" * 50)
    print("🔧 Notebook State Repair")
    print("=" * 50)
    
    # 1. Memory cleanup
    gc.collect()
    print("✅ Memory cleanup completed")
    
    # 2. Check kernel state
    print(f"🐍 Python version: {sys.version}")
    print(f"💾 Memory usage: {sys.getsizeof(globals())} bytes")
    
    # 3. Check variable state
    important_vars = ['MODEL_PATH', 'EXP_NAME', 'model', 'device', 'config']
    for var in important_vars:
        if var in globals():
            print(f"✅ {var}: Defined")
        else:
            print(f"❌ {var}: Not defined")
    
    print("\n🎯 Repair suggestions:")
    print("1. If cell background is dimmed, double-click the cell to enter edit mode")
    print("2. Press Ctrl+Z to undo recent modifications")
    print("3. Press Ctrl+Shift+P and search for 'Reload Window'")
    print("4. If issues persist, restart kernel (Ctrl+Shift+P -> 'Restart Kernel')")
    
    # JavaScript to fix frontend display issues
    display(Javascript("""
    console.log('Fixing cell state...');
    
    // Remove possible locked state
    document.querySelectorAll('.cell').forEach(cell => {
        cell.classList.remove('readonly', 'locked', 'disabled');
        cell.style.opacity = '1';
        cell.style.backgroundColor = '';
    });
    
    // Reactivate edit functionality
    document.querySelectorAll('.jp-Cell').forEach(cell => {
        cell.style.opacity = '1';
        cell.style.filter = 'none';
    });
    
    console.log('Cell state repair completed');
    """))

# ===========================
# 3. MAIN TESTING EXECUTION
# ===========================

def run_comprehensive_cpu_tests():
    """Run all CPU tests comprehensively"""
    print("🚀 Starting Comprehensive CPU Testing Suite")
    print("=" * 60)
    
    # Verify required variables exist
    try:
        print(f"✅ Using model path: {MODEL_PATH}")
        print(f"✅ Using experiment config: {EXP_NAME}")
    except NameError:
        print("❌ Please run the model path configuration cell first!")
        return
    
    # Load configuration
    try:
        with open("config/data_config.yaml", "r") as f:
            default_config = yaml.safe_load(f)
        config = default_config
        
        with open(f'config/{EXP_NAME}.yaml', "r") as f:
            user_config = yaml.safe_load(f)
        config.update(user_config)
        latent_size = config['image_size'] // 8
        
        print(f"✅ Configuration loaded successfully")
        
    except Exception as e:
        print(f"❌ Configuration loading failed: {e}")
        return
    
    # Create mock models
    print("\n🏗️  Creating mock models for CPU testing...")
    
    global model, diffusion, vae, device
    
    model = MockHybridModel(
        latent_size=latent_size,
        memory_enabled=config.get('memory_enabled', True),
        memory_buffer_size=config.get('memory_buffer_size', 100)
    )
    
    device = 'cpu'
    model.to(device)
    model.reset_memory()
    
    diffusion = MockDiffusion()
    vae = MockVAE(device)
    
    print("✅ Mock models created successfully")
    
    # Run all tests
    test_results = {}
    
    try:
        test_results['file_structure'] = test_file_structure()
        test_results['config_loading'] = test_config_loading()
        test_results['memory_storage'] = test_memory_storage_mechanism()
        test_results['memory_retrieval'] = test_memory_retrieval_criteria()
        test_results['integration'] = test_integration()
        
        # Additional demonstrations
        navigation_demo = create_mock_navigation_demo()
        
        # Final summary
        print("\n" + "=" * 60)
        print("📋 COMPREHENSIVE TEST SUMMARY")
        print("=" * 60)
        
        print("✅ Passed tests:")
        if all(test_results['file_structure'].values()):
            print("   - File structure validation")
        if all(test_results['config_loading'].values()):
            print("   - Configuration file loading")
        if test_results['memory_storage']['frames_added'] > 0:
            print("   - Memory storage mechanism")
        if test_results['memory_retrieval']:
            print("   - Memory retrieval criteria")
        if test_results['integration']:
            print("   - System integration")
        
        print("\n⚠️  Issues requiring attention:")
        failed_files = [k for k, v in test_results['file_structure'].items() if not v]
        if failed_files:
            print(f"   - Missing files/directories: {', '.join(failed_files)}")
        
        failed_configs = [k for k, v in test_results['config_loading'].items() if not v]
        if failed_configs:
            print(f"   - Configuration loading failed: {', '.join(failed_configs)}")
        
        # Repair utilities
        fix_notebook_state()
        
        print("\n🎉 Comprehensive testing completed!")
        print("💡 Note: This is a simulation environment, actual GPU environment may perform differently")
        
        return test_results
        
    except Exception as e:
        print(f"❌ Testing failed with error: {e}")
        import traceback
        traceback.print_exc()
        return None

# Run the comprehensive test suite
if __name__ == "__main__" or True:  # Allow execution in notebook
    test_results = run_comprehensive_cpu_tests()